# settings, Download modules

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import torch
torch.cuda.empty_cache()

In [4]:
# Install transformers library.
!pip install -q git+https://github.com/huggingface/transformers.git
# Install helper functions.
!pip install -q git+https://github.com/gmihaila/ml_things.git
# Install matplotlib in proper version
! pip install matplotlib==3.1.3

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 6.6 MB 4.2 MB/s 
     |████████████████████████████████| 596 kB 44.2 MB/s 
     |████████████████████████████████| 86 kB 5.5 MB/s 
     |████████████████████████████████| 53 kB 1.2 MB/s 
     |████████████████████████████████| 11.2 MB 7.5 MB/s 
     |████████████████████████████████| 930 kB 42.3 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 13.1 MB 3.6 MB/s 
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.5.2
    Uninstal

In [5]:
# Install wandb
! pip install wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.8 MB 4.2 MB/s 
     |████████████████████████████████| 145 kB 55.8 MB/s 
     |████████████████████████████████| 181 kB 53.9 MB/s 
     |████████████████████████████████| 63 kB 1.6 MB/s 
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=da7fe38257b0e7ad8f8ea04092ddd47343e25a6b84f11970f7cec09a066261ee
  Stored in directory: /root/.cache/pip/wheels/3e/31/09/fa59cef12cdcfecc627b3d24273699f390e71828921b2cbba2
Successfully built pathtools


In [6]:
import matplotlib.pyplot as plt

plt.rc('font', family='NanumBarunGothic') 

## load_data.py

In [7]:
import pickle as pickle
import os
import pandas as pd
import torch

class news_dataset(torch.utils.data.Dataset):
    """Dataset 구성을 위한 class."""

    def __init__(self, news_dataset, labels):
        self.news_dataset = news_dataset
        self.labels = labels

    def __getitem__(self, idx):
        item = {
            key: val[idx].clone().detach() for key, val in self.news_dataset.items()
        }
        item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)


def load_data(dataset_dir):
    """csv 파일을 경로에 맡게 불러 옵니다."""
    pd_dataset = pd.read_csv(dataset_dir)
    return pd_dataset


def tokenized_dataset(dataset, tokenizer, max_length):
    """tokenizer에 따라 sentence를 tokenizing 합니다."""
    concat_entity = []
    for temp in dataset['conversation']:
        concat_entity.append(temp)

    tokenized_sentences = tokenizer(
        concat_entity,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=max_length,
        add_special_tokens=True,
        return_token_type_ids=False, #BERT ->  RoBERTa 120, 20 100 밥을 먹었다 [PAD] [PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD] [1 1 1 00000000]
    )
    # Tokenizer : sentence -> token_id , attention mask , token_type_ids => 첫번째 문장과 두번째 문장 표시 00000000 111111111
    return tokenized_sentences

## train.py

In [14]:
! wandb login

wandb: Currently logged in as: ethicsense. Use `wandb login --relogin` to force relogin


In [ ]:
# 모델을 실험을 해요. 기록 & 저장 -> WandB , MLflow / 실험하기 용이 -> argparse : python train.py -b 32 -lr 5e-5 + 전기세  nohup
# config.json

import os
import random
import argparse

import numpy as np
from sklearn.metrics import accuracy_score

import torch
from torch.utils.data import random_split
from transformers import (
    AutoTokenizer,
    AutoConfig,
    AutoModelForSequenceClassification,
)

from transformers import Trainer, TrainingArguments
from transformers import EarlyStoppingCallback
from transformers.optimization import get_cosine_with_hard_restarts_schedule_with_warmup

import wandb

# ------* Fix Seeds * -----------#
def seed_everything(seed):
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  # if use multi-GPU
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    random.seed(seed)


def compute_metrics(pred):
    """validation을 위한 metrics function"""
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1) #argmax([0.01 0.0001 0.1])

    # calculate accuracy using sklearn's function
    acc = accuracy_score(labels, preds)
    return {
        "accuracy": acc,
    }


def label_to_num(label):
    dict_label_to_num = {
        '일반':0, '갈취':1, '협박':2, '직장 내 괴롭힘':3, '기타 괴롭힘':4
    }
    num_label = []

    for v in label:
        num_label.append(dict_label_to_num[v])

    return num_label


def train():
    # fix a seed
    seed_everything(seed)

    # load model and tokenizer
    MODEL_NAME = model_name # klue/roberta-base
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

    # load dataset
    org_dataset = load_data('./drive/MyDrive/Colab_Notebooks/voc_rm/datasets/dktc/data/insert_koen/train.csv')
    # test_dataset = load_data("..data/newszum_test_data.csv")

    org_label = label_to_num(org_dataset["class"].values)
    # test_label = label_to_num(test_dataset["class"].values)

    # tokenizing dataset
    tokenized_corpus = tokenized_dataset(org_dataset, tokenizer, max_len)
    # tokenized_test = tokenized_dataset(test_dataset, tokenizer, max_len)

    # make dataset for pytorch.
    voc_dataset = news_dataset(tokenized_corpus, org_label)
    # news_test_dataset = news_dataset(tokenized_test, test_label)

    # defining split_size
    dataset_size = len(voc_dataset)
    train_size = int(dataset_size * 0.8)
    validation_size = int(dataset_size * 0.1)
    test_size = dataset_size - train_size - validation_size

    # random split
    train_dataset, valid_dataset, test_dataset = random_split(voc_dataset, [train_size, validation_size, test_size])

    print(f"Training Data Size : {len(train_dataset)}")
    print(f"Validation Data Size : {len(valid_dataset)}")
    print(f"Testing Data Size : {len(test_dataset)}")
    print()
    print()
    train_dataset.__getitem__(0)

    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print(device) #a.device ,b => a*b XX

    # setting model hyperparameter
    model_config = AutoConfig.from_pretrained(MODEL_NAME)
    model_config.num_labels = 5

    model = AutoModelForSequenceClassification.from_pretrained(
        MODEL_NAME, config=model_config
    )

    print(model.config)
    model.parameters
    model.to(device)

    ### callback & optimizer & scheduler 추가
    MyCallback = EarlyStoppingCallback(
        early_stopping_patience=3, early_stopping_threshold=0.001
    )

    optimizer = torch.optim.AdamW(
        model.parameters(),
        lr=lr,
        betas=(0.9, 0.999),
        eps=1e-08,
        weight_decay=weight_decay,
        amsgrad=False,
    )

    training_args = TrainingArguments(
      output_dir = save_path + '/results',
      save_total_limit = save_limit,
      save_steps = save_step,
      num_train_epochs = epochs,
      learning_rate = lr,
      per_device_train_batch_size = batch_size,
      per_device_eval_batch_size = 2,
      warmup_steps = warmup_steps,
      weight_decay = weight_decay,
      logging_dir = save_path + '/logs',
      logging_steps = 100,
      evaluation_strategy = 'steps',
      gradient_accumulation_steps = 20,
      eval_accumulation_steps = 20,

      eval_steps = 500,
      load_best_model_at_end = True,
    )

    trainer = Trainer(
      model = model,
      args = training_args,
      train_dataset=train_dataset,  # training dataset
      eval_dataset=valid_dataset,  # evaluation dataset
      compute_metrics=compute_metrics,  # define metrics function
      callbacks=[MyCallback],
      optimizers=(
        optimizer,
        get_cosine_with_hard_restarts_schedule_with_warmup(
            optimizer,
            num_warmup_steps=warmup_steps,
            num_training_steps=len(train_dataset) * epochs,
            ),
        ),
    )

    # train model
    trainer.train()
    model.save_pretrained(save_path + "/best_model")

def main():
    train()


if __name__ == "__main__":
    save_path = './drive/MyDrive/Colab_Notebooks/voc_rm/model/bert'

    model_name = "klue/roberta-base"
    seed = 42
    max_len = 400
    save_limit = 5
    save_step = 500
    epochs = 10
    lr = 5e-3
    batch_size = 4
    per_device_eval_batch_size = 2
    warmup_steps = 300
    weight_decay = 0.01
    logging_steps = 100

    eval_steps = 500
    load_best_model_at_end = True

    # fix a seed
    seed_everything(seed)

    wandb.config = {
        'epochs': 10,
        'classes': 5,
        'batch_size': 4,
        'kernels': [16, 32],
        'weight_decay': 0.01,
        'learning_rate': 5e-3,
        'dataset': 'DKTC',
        'architecture': 'RoBERTa',
        'seed': 42
    }
    wandb.init(project="huggingface", entity="ethicsense")
    main()

loading file https://huggingface.co/klue/roberta-base/resolve/main/vocab.txt from cache at /root/.cache/huggingface/transformers/e8441a174492958462b6b16b6db8f1e7253cd149ca779522cadd812d55091b89.d1b86bed49516351c7bb29b19d7e7be2ab53b931bcb1f9b2aacfb71f2124d25a
loading file https://huggingface.co/klue/roberta-base/resolve/main/tokenizer.json from cache at /root/.cache/huggingface/transformers/233a5b2c17873a8477b62dd92a02092a9937759e924a5f22b111becebb8aba5e.44c30ade4958fcfd446e66025e10a5b380cdd0bbe9b3fb7a794f357e7f0f34c2
loading file https://huggingface.co/klue/roberta-base/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/klue/roberta-base/resolve/main/special_tokens_map.json from cache at /root/.cache/huggingface/transformers/9d0c87e44b00acfbfbae931b2e4068eb6311a0c3e71e23e5400bdf57cab4bfbf.70c17d6e4d492c8f24f5bb97ab56c7f272e947112c6faf9dd846da42ba13eb23
loading file https://huggingface.co/klue/roberta-base/resolve/main/tokenizer_config.json from cache 

Training Data Size : 6319
Validation Data Size : 789
Testing Data Size : 791


cpu


loading configuration file https://huggingface.co/klue/roberta-base/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/a96469ca2a238496d435a0e9e202f261119c146a0326444b6d68ae1adc35e04f.85b0b02ba2a483f3adb8a60ab70dbd875768fcd5e6cdb21a593c6e02fdffac3a
Model config RobertaConfig {
  "_name_or_path": "klue/roberta-base",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "tokenizer_class": "BertTokenizer",
  "transformers_version": "4.20.0.dev0",
  "type_vocab_size": 1,
  "use_cache": t

RobertaConfig {
  "_name_or_path": "klue/roberta-base",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4
  },
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "tokenizer_class": "BertTokenizer",
  "transformers_version": "4.20.0.dev0",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 32000
}



## inference.py

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from torch.utils.data import DataLoader
import pandas as pd
import torch
import torch.nn.functional as F

import numpy as np
import argparse
from tqdm import tqdm


def inference(model, tokenized_sent, device):
    """
    test dataset을 DataLoader로 만들어 준 후,
    batch_size로 나눠 model이 예측 합니다.
    """
    dataloader = DataLoader(tokenized_sent, batch_size=16, shuffle=False)
    model.eval()
    output_pred = []
    for i, data in enumerate(tqdm(dataloader)):
        with torch.no_grad():
            outputs = model(
                input_ids=data["input_ids"].to(device),
                attention_mask=data["attention_mask"].to(device),
            )
        # print(outputs[0])
        logits = outputs[0]
        logits = logits.detach().cpu().numpy()
        result = np.argmax(logits, axis=-1)

        output_pred.append(result)
    return (np.concatenate(output_pred).tolist(),)


def num_to_label(label):
    """
    숫자로 되어 있던 class를 원본 문자열 라벨로 변환 합니다.
    """
    origin_label = []
    dict_num_to_label = {
        '일반':0, '갈취':1, '협박':2, '직장 내 괴롭힘':3, '기타 괴롭힘':4
    }

    for v in label[0]:
        origin_label.append(dict_num_to_label[v])

    return origin_label


def load_test_dataset(dataset_dir, tokenizer):
    """
    test dataset을 불러온 후,
    tokenizing 합니다.
    """
    test_dataset = load_data(dataset_dir)
    test_label = list(map(int, label_to_num(test_dataset["class"].values)))

    # tokenizing dataset
    tokenized_test = tokenized_dataset(test_dataset, tokenizer, 384)
    return tokenized_test, test_label


def main():
    """
    주어진 dataset csv 파일과 같은 형태일 경우 inference 가능한 코드입니다.
    """
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    # load tokenizer
    Tokenizer_NAME = model
    tokenizer = AutoTokenizer.from_pretrained(Tokenizer_NAME)

    ## load my model
    MODEL_NAME = model_dir  # model dir.

    model = AutoModelForSequenceClassification.from_pretrained(model_dir)
    # model.parameters
    model.to(device)

    # ## load test datset
    # test_dataset_dir = "..data/newszum_test_data.csv"
    # test_dataset, test_label = load_test_dataset(test_dataset_dir, tokenizer)
    # news_test_dataset = news_dataset(test_dataset, test_label)

    ## predict answer
    pred_answer = inference(model, test_dataset, device)  # model에서 class 추론
    pred_answer = num_to_label(pred_answer)
    # test_dataset = load_data(test_dataset_dir)

    # ## make csv file with predicted answer
    # #########################################################
    # output = pd.DataFrame(
    #     {
    #         "title": test_dataset["title"],
    #         "cleanBody": test_dataset["cleanBody"],
    #         "category": list(test_dataset["category"].values),
    #         "result": pred_answer,
    #     }
    # )

    # output.to_csv(
    #     "./prediction/submission.csv", index=False
    # )  # 최종적으로 완성된 예측한 라벨 csv 파일 형태로 저장.
    # print("---- Finish! ----")


if __name__ == "__main__":

    # model dir
    model = 'klue/roberta-base'
    model_dir = save_path + '/best_model'
    main()